In [4]:

import os



In [41]:
# Create table
import sqlite3

with sqlite3.connect('./db.sqlite') as con:
    with open('./schema.sql') as f:
        con.execute("DROP TABLE IF EXISTS terms;")
        con.execute("DROP TABLE IF EXISTS courses;")
        con.executescript(f.read())

In [42]:
import os
import time
import openapi_client
from openapi_client.rest import ApiException
from pprint import pprint
import sqlite3

# Defining the host is optional and defaults to http://localhost
# See configuration.py for a list of all supported configuration parameters.
configuration = openapi_client.Configuration(
    host="https://openapi.data.uwaterloo.ca",
)

# The client must configure the authentication and authorization parameters
# in accordance with the API server security policy.
# Examples for each auth method are provided below, use the example that
# satisfies your auth use case.

# Configure API key authorization: apiKey
configuration.api_key['apiKey'] = os.environ["API_KEY"]

# Uncomment below to setup prefix (e.g. Bearer) for API key, if needed
# configuration.api_key_prefix['apiKey'] = 'Bearer'


# CREATE TABLE terms
# (
#     id                     INTEGER PRIMARY KEY AUTOINCREMENT,
#     termCode               TEXT     NOT NULL,
#     name                   TEXT     NOT NULL,
#     nameShort              TEXT     NOT NULL,
#     termBeginDate          DATE     NOT NULL,
#     termEndDate            DATE     NOT NULL,
#     sixtyPercentDate       DATE     NOT NULL,
#     associatedAcademicYear INTEGER  NOT NULL,
#     created_at             DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
#     updated_at             DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP
# );

with sqlite3.connect('./db.sqlite') as con:
    cur = con.cursor()
    # Enter a context with an instance of the API client
    with openapi_client.ApiClient(configuration) as api_client:
        # Create an instance of the API class
        api_instance = openapi_client.TermsApi(api_client)

        try:
            # Gets all Academic Organization data
            api_response = api_instance.v3_terms_get()

            for term in api_response:
                cur.execute("""
                    INSERT INTO terms (termCode, name, nameShort, termBeginDate, termEndDate, sixtyPercentDate, associatedAcademicYear)
                    VALUES (?, ?, ?,?, ?, ?,?)
                """, (term.term_code, term.name, term.name_short, term.term_begin_date, term.term_end_date,
                      term.sixty_percent_complete_date, term.associated_academic_year))
            pprint(api_response)
        except ApiException as e:
            print("Exception when calling AcademicOrganizationsApi->v3_academic_organizations_get: %s\n" % e)



[Term(term_code='1189', name='Fall 2018', name_short='2018 F', term_begin_date=datetime.datetime(2018, 9, 1, 0, 0), term_end_date=datetime.datetime(2018, 12, 31, 0, 0), sixty_percent_complete_date=datetime.datetime(2018, 11, 1, 0, 0), associated_academic_year=2019),
 Term(term_code='1209', name='Fall 2020', name_short='2020 F', term_begin_date=datetime.datetime(2020, 9, 1, 0, 0), term_end_date=datetime.datetime(2020, 12, 31, 0, 0), sixty_percent_complete_date=datetime.datetime(2020, 11, 1, 0, 0), associated_academic_year=2021),
 Term(term_code='1169', name='Fall 2016', name_short='2016 F', term_begin_date=datetime.datetime(2016, 9, 1, 0, 0), term_end_date=datetime.datetime(2016, 12, 31, 0, 0), sixty_percent_complete_date=datetime.datetime(2016, 11, 1, 0, 0), associated_academic_year=2017),
 Term(term_code='1165', name='Spring 2016', name_short='2016 S', term_begin_date=datetime.datetime(2016, 5, 1, 0, 0), term_end_date=datetime.datetime(2016, 8, 31, 0, 0), sixty_percent_complete_date=d

In [44]:

# CREATE TABLE courses (
#     id                          INTEGER PRIMARY KEY AUTOINCREMENT,
#     courseId                    TEXT    NOT NULL,
#     courseOfferNumber           INTEGER NOT NULL,
#     termCode                    TEXT    NOT NULL,
#     termName                    TEXT    NOT NULL,
#     associatedAcademicCareer    TEXT    NOT NULL,
#     associatedAcademicGroupCode TEXT    NOT NULL,
#     associatedAcademicOrgCode   TEXT    NOT NULL,
#     subjectCode                 TEXT    NOT NULL,
#     catalogNumber               TEXT    NOT NULL,
#     title                       TEXT    NOT NULL,
#     descriptionAbbreviated      TEXT    NOT NULL,
#     description                 TEXT    NOT NULL,
#     gradingBasis                TEXT    NOT NULL,
#     courseComponentCode         TEXT    NOT NULL,
#     enrollConsentCode           TEXT    NOT NULL,
#     enrollConsentDescription    TEXT    NOT NULL,
#     dropConsentCode             TEXT    NOT NULL,
#     dropConsentDescription      TEXT    NOT NULL,
#     requirementsDescription     TEXT    NOT NULL,
#     created_at                  DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
#     updated_at                  DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP
# );


with sqlite3.connect('./db.sqlite') as con:
    cur = con.cursor()
    # Enter a context with an instance of the API client
    with openapi_client.ApiClient(configuration) as api_client:
        # Create an instance of the API class
        api_instance = openapi_client.CoursesApi(api_client)

        # For each term
        cur.execute("SELECT termCode FROM terms")
        term_codes = [term[0] for term in cur.fetchall()]
        # term_codes = ['1231', '1241']

        for term_code in term_codes:
            print("Trying term: " + term_code)
            try:
                api_response = api_instance.v3_courses_term_code_get(term_code=term_code)
                for course in api_response:
                    cur.execute("""
                        INSERT INTO courses (courseId, courseOfferNumber, termCode, termName, associatedAcademicCareer, associatedAcademicGroupCode, associatedAcademicOrgCode, subjectCode, catalogNumber, title, descriptionAbbreviated, description, gradingBasis, courseComponentCode, enrollConsentCode, enrollConsentDescription, dropConsentCode, dropConsentDescription, requirementsDescription)
                         VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)

                        """, (course.course_id, course.course_offer_number, course.term_code, course.term_name,
                              course.associated_academic_career, course.associated_academic_group_code,
                              course.associated_academic_org_code, course.subject_code, course.catalog_number,
                              course.title, course.description_abbreviated, course.description, course.grading_basis,
                              course.course_component_code, course.enroll_consent_code,
                              course.enroll_consent_description, course.drop_consent_code,
                              course.drop_consent_description, course.requirements_description))

            except ApiException as e:
                print(f"Failed to get course for term {term_code}.  {e}\n")



Failed to get course for term 0561.  (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Transfer-Encoding': 'chunked', 'Content-Type': 'application/problem+json; charset=utf-8', 'Server': 'Microsoft-IIS/10.0', 'Strict-Transport-Security': 'max-age=2592000', 'X-Powered-By': 'ASP.NET', 'Date': 'Mon, 15 Jan 2024 01:06:30 GMT'})
HTTP response body: {"type":"https://tools.ietf.org/html/rfc7231#section-6.5.4","title":"Not Found","status":404,"traceId":"00-9513b51da962e07872c0decaee1db13a-bdd23d1bfa01c655-00"}


Failed to get course for term 0565.  (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Transfer-Encoding': 'chunked', 'Content-Type': 'application/problem+json; charset=utf-8', 'Server': 'Microsoft-IIS/10.0', 'Strict-Transport-Security': 'max-age=2592000', 'X-Powered-By': 'ASP.NET', 'Date': 'Mon, 15 Jan 2024 01:06:30 GMT'})
HTTP response body: {"type":"https://tools.ietf.org/html/rfc7231#section-6.5.4","title":"Not Found","status":404,"traceId":"00-f43559fde